In [ ]:
# !pip install langchain
# pip install google-api-python-client

In [ ]:
import googleapiclient.discovery
import googleapiclient.errors,re

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = "Developer-API-Key"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

def extract_video_id(url):
    # Regular expression to find the video ID
    video_id = None
    patterns = [
        r'^https?://(?:youtu\.be/|www\.youtube\.com(?:/embed/|/v/|/watch\?v=|/watch\?.+&v=))([\w-]{11})(?:$|&|\?)',
        r'^https?://(?:www\.youtube-nocookie\.com/v/|www\.youtube\.com/v/|youtu\.be/)([\w-]{11})']

    for pattern in patterns:
        match = re.search(pattern, url.strip())
        if match:
            video_id = match.group(1)
            break

    return video_id

comments = []

# Function to fetch comments
def fetch_comments(youtube, video_id):
    video_id = extract_video_id(video_id)
    if video_id:
        next_page_token = None
        while True:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                textFormat="plainText",
                maxResults=100,
                pageToken=next_page_token)
            response = request.execute()

            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                comments.append(comment)

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break
    else:
        print("Invalid URL.")

video_url = "htsdftps://youtu.be/UrsmFxEIp5k?si=8tC3sewys-diVnEa"
fetch_comments(youtube, video_url)

In [ ]:
from transformers import pipeline
pipe = pipeline("text-classification", model="ProsusAI/finbert",truncation=True)
from langchain_core.runnables import RunnablePassthrough

def get_sentiment(comment):
    res = pipe(comment)
    return {comment:res[0]['label']}

def custom_output_parser(data):
    comments_with_sentiments = {}
    for item in data:
      for comment,sentiment in item.items():
        comments_with_sentiments[comment] = sentiment
    return comments_with_sentiments

chain = RunnablePassthrough() | get_sentiment

res = custom_output_parser(await chain.abatch(comments))
print(res)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.DataFrame(list(res.items()),columns=["comments","sentiments"])

print(df["sentiments"].value_counts())

plt.figure(figsize=(10,5))
plt.bar(df["sentiments"].value_counts().index,df["sentiments"].value_counts().values)
plt.show()